In [1]:
# Original filename: "02_ncifp_flow_reqts_v2.py"

In [47]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt

In [48]:
# Paths, input, and output file names

outer_dir = os.path.abspath(os.path.join(os.getcwd() ,"../.."))
refpth = os.path.join(outer_dir, 'IFT_files', 'Reference Files')
sfe_char_csv = os.path.join(refpth, 'TNC SFE LOI Characteristics with MAF.csv')
sfe_char_xls= os.path.join(refpth, 'TNC SFE LOI Characteristics.xlsx')
subset_xls = os.path.join(refpth, 'TNC Subset SFE LOIs.xlsx')
#sfe_char_csv = os.path.join(refpth, 'All SFE LOI Characteristics with MAF.csv')
#sfe_char_xls= os.path.join(refpth, 'All SFE LOI Characteristics.xlsx')
#subset_xls = os.path.join(refpth, 'Subset SFE LOIs.xlsx')
unimpath = os.path.join(outer_dir, "IFT_files", 'Unimpaired Flow')

startdir = os.path.join(outer_dir, 'IFT_files', 'IFT Results')
alldates = pd.date_range(dt.datetime(1950,10,1),dt.datetime(2021,9,30))
out_csv = os.path.join(startdir, 'All LOI NCIFP IFTs_TNC.csv')

In [49]:
# Original filename: "00_read_loi_paradigm_flow_v3.py"
def read_loi_paradigm_flow(p):
    unimpflowfile = os.path.join(unimpath, str(p) + '.csv')
    unimp = pd.read_csv(unimpflowfile,index_col=0)
    unimp.index = pd.DatetimeIndex(unimp.index)
    return unimp

In [50]:
# Original filename: "00_add_ts_col.py"
def add_ts_col(tab):
#adds column for TS as first column of DataFrame as formatted date
    tab['TS'] = tab.index.strftime('%m/%d/%Y')
    cols = tab.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    tab = tab[cols]
    return tab

In [51]:
# Calculate bankful flow (Qbf) using the 1.5 year recurrance interval according to NCIFP
def rec_int_flow(p,ri):
    '''Function to estimate the closest max flow value for a given recurrence interval'''
    df_daily = read_loi_paradigm_flow(p)
    # Calculate max flow by water year
    df = (df_daily.groupby('Water Year')
               .agg({'flow':'max', 'Day':'count'})
               .reset_index()
               .rename(columns={'flow':'max_flow', 'Day':'day_count'})
              )
    # Remove partial years
    df = df.loc[df['day_count'] >= 365]
    # Calculate recurrence interval
    df = df.sort_values(by=['max_flow'], ascending=[False]).reset_index()
    df['flow_rank'] = df['max_flow'].rank(ascending = False)
    num_years = len(df)
    df['prob_exceedence'] = df.flow_rank / (num_years + 1)
    df['recurrence_interval'] = (num_years + 1) / df.flow_rank
    # Return flow for specified recurrence interval
    row_num = df['recurrence_interval'].sub(ri).abs().idxmin()
    Q = df.iloc[row_num]['max_flow']
    return Q


In [52]:
# Original filename: "00_get_all_sfe_lois.py"
# Read in SFE LOI characteristics table and calculate bankfull flow using cont. Area relation

def get_all_sfe_lois():
    sfelois = pd.read_csv(sfe_char_csv,index_col=0)
    loi = [str(i) for i in sfelois['LOI']]
    sfelois['Outlet LOI'] = sfelois['LOI']
    sfelois['Contributing Area (mi^2)'] = sfelois['Contributing Area']
    sfelois['MAF'] = sfelois['Mean Annual Flow (cfs)']
    #sfelois['Qbf'] = 71.5 * sfelois['Contributing Area (mi^2)'] #71.5 cfs/mi^2 according to Darren
    sfelois['Qbf'] = sfelois.apply(lambda row : rec_int_flow(str(row['LOI']),1.5), axis = 1)
    subset = pd.read_excel(subset_xls)
    sublois = sfelois.loc[subset['SWSID'],:]
    return sublois, loi, sfelois

In [53]:
loitab, loi, fultab = get_all_sfe_lois()
loitab = fultab

In [54]:
loitab

,LOI,dnLOI,Lat,Lon,Contributing Area,Geomorphic Class,Geologic Setting,Mean Annual Flow (cfs),COMID,Outlet LOI,Contributing Area (mi^2),MAF,Qbf
2664961,Indian,NaN,NaN,NaN,39.486536,NaN,NaN,63.202346,2664961,Indian,39.486536,63.202346,1105.039921
8285498,Sproul,NaN,NaN,NaN,4.992762,NaN,NaN,17.108696,8285498,Sproul,4.992762,17.108696,201.685417


In [55]:
dailyreqts = pd.DataFrame(index=alldates,columns=loi)  #set up IFT table for WEAP
loitab['NCIFP MBF'] = np.zeros_like(loitab['MAF']) #initialize minimum bypass flow metric

In [56]:
for l in loi: #loop through lcoations
    loimaf = loitab.loc[loitab.LOI == l,'MAF'].values[0] #extract LOI mean annual flow
    loicta = loitab.loc[loitab.LOI == l,'Contributing Area (mi^2)'].values[0] #extract LOI contributing area
    loiqbf = loitab.loc[loitab.LOI == l,'Qbf'].values[0] #extract LOI bankfull flow
    loistr = l
    unimp = read_loi_paradigm_flow(loistr)
    #NCIFP calculations based on bin of Contributing area
    if loicta <= 1:
        loimbf = 9 * loimaf
    elif (loicta > 1) & (loicta < 321):
        loimbf = 8.8 * loimaf * loicta ** -0.47
    else:
        loimbf = 0.6 * loimaf
    print("MBF for {} is: {}".format(l,loimbf))
    loidiv = loiqbf*0.05 #diversion allocation is 5% of bankfull
    loitab.loc[loitab.LOI == l,'NCIFP MBF'] = loimbf #record MBF
    unimp['NCIFP div aloc'] = np.zeros_like(unimp['flow'])
    unimp['flow above mbf'] = unimp['flow']-loimbf #calculate flow above MBF
    unimp.loc[unimp['flow above mbf']>loidiv,'flow above mbf'] = loidiv #allow full diversion amount when there is more than enough flow to divert
    #get index of diversion period
    divinds = (unimp['flow'] > loimbf) & ((unimp.index.month <= 3) | ((unimp.index.day >= 15) & (unimp.index.month == 12)))
    unimp.loc[divinds,'NCIFP div aloc'] = unimp.loc[divinds,'flow above mbf'] #loidiv
    unimp['NCIFP flow req'] = unimp['flow'] - unimp['NCIFP div aloc'] #IFT = flow - diversion
    dailyreqts.loc[:,loistr] = unimp.loc[dailyreqts.index,'NCIFP flow req']

MBF for Indian is: 98.82909621545929
MBF for Sproul is: 70.7097621188737


d:\Users\kklausmeyer\AppData\Local\Continuum\anaconda3\envs\freshwater\lib\site-packages\pandas\core\indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]
d:\Users\kklausmeyer\AppData\Local\Continuum\anaconda3\envs\freshwater\lib\site-packages\pandas\core\indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [57]:
unimp

,Year,Month,Day,Water Year,flow,NCIFP div aloc,flow above mbf,NCIFP flow req
DTTM,,,,,,,,
1960-10-01,1960,10,1,1961,0.692813,0.0,-70.016950,0.692813
1960-10-02,1960,10,2,1961,0.692813,0.0,-70.016950,0.692813
1960-10-03,1960,10,3,1961,0.692813,0.0,-70.016950,0.692813
1960-10-04,1960,10,4,1961,0.692813,0.0,-70.016950,0.692813
1960-10-05,1960,10,5,1961,0.692813,0.0,-70.016950,0.692813
1960-10-06,1960,10,6,1961,0.692813,0.0,-70.016950,0.692813
1960-10-07,1960,10,7,1961,0.692813,0.0,-70.016950,0.692813
1960-10-08,1960,10,8,1961,0.692813,0.0,-70.016950,0.692813
1960-10-09,1960,10,9,1961,0.692813,0.0,-70.016950,0.692813


In [58]:
dailyreqts = add_ts_col(dailyreqts)
dailyreqts.to_csv(out_csv)